In [1]:
import sys
sys.path.append("./utils/") # Adds higher directory to python modules path.
import random
import numpy as np
import cv2
import yaml
import glob, os
import loadcoraldata_utils as coralutils
import json
import importlib
from NeMO_generator import NeMOImageGenerator, ImageSetLoader
from osgeo import gdal
from sklearn.manifold import TSNE

from matplotlib import colors
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
%matplotlib notebook

c:\users\rechant\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Fill in truth maps as taken from Jarrett's submissions
imagepath = "F:/Images/processed_Fiji_files"
startpath = "../Images/submissions 7-24-18"
finalpath_RGB = "../Images/Jarrett_submissions2/Original"
finalpath_patch = "../Images/Jarrett_submissions2/Patches"
finalpath_truthmap = "../Images/Jarrett_submissions2/Truthmaps_RGB"
rasterfile = startpath + "/rastertrain.txt"
print(rasterfile)
image_size = 256
offset = 128

alldir = [os.path.join(startpath,o) for o in os.listdir(startpath) if os.path.isdir(os.path.join(startpath,o))]
counter = 0
for d in alldir:
    files = [f for f in os.listdir(d) if os.path.isfile(os.path.join(d,f))]
    for f in files:
        fillmap = coralutils.fill_in_truthmap(os.path.join(d,f), 3, nofillcolor=np.asarray([0,0,0]))
        filestr = "Coral_" + str(counter).zfill(8) + ".png"
        cv2.imwrite(os.path.join(finalpath_truthmap,filestr), fillmap)
        if f.endswith(".tiff"):
            temp_f = f[:-1] # turn .tiff to .tif
        else:
            temp_f = f
        
        filestr = "Coral_" + str(counter).zfill(8) + ".tif"
        patch, patch_proj, patch_gt = coralutils.load_specific_patch(imagepath, temp_f, rasterfile, image_size, offset=offset)
        patch_R = 255/200*patch[:,:,4]
        patch_G = 255/200*patch[:,:,2]
        patch_B = 255/200*patch[:,:,1]
        patch_R[patch_R > 255] = 255
        patch_G[patch_G > 255] = 255
        patch_B[patch_B > 255] = 255
        patch_BGR = np.rollaxis(np.asarray([patch_B, patch_G, patch_R], dtype=np.uint8),0,3)
        cv2.imwrite(os.path.join(finalpath_RGB,filestr), patch_BGR)
        
        driver = gdal.GetDriverByName('GTiff')
        dataset = driver.Create(finalpath_patch + "/" + filestr, image_size, image_size, patch.shape[2], gdal.GDT_Float32)
        for chan in range(patch.shape[2]):
            dataset.GetRasterBand(chan+1).WriteArray((patch[:,:,chan]-100)/100)
            dataset.FlushCache()
        
        counter += 1
        print("Finished output for " + str(counter) + " images")

../Images/submissions 7-24-18/rastertrain.txt
Finished output for 1 images
Finished output for 2 images
Finished output for 3 images
Finished output for 4 images
Finished output for 5 images
Finished output for 6 images
Finished output for 7 images
Finished output for 8 images
Finished output for 9 images
Finished output for 10 images
Finished output for 11 images
Finished output for 12 images
Finished output for 13 images
Finished output for 14 images
Finished output for 15 images
Finished output for 16 images
Finished output for 17 images
Finished output for 18 images
Finished output for 19 images
Finished output for 20 images
Finished output for 21 images
Finished output for 22 images
Finished output for 23 images
Finished output for 24 images
Finished output for 25 images
Finished output for 26 images
Finished output for 27 images
Finished output for 28 images
Finished output for 29 images
Finished output for 30 images
Finished output for 31 images
Finished output for 32 images
Fin

In [4]:
jsonpath = './utils/CoralClasses.json'
finalpath_truthmap = "../Images/Jarrett_submissions2/Truthmaps_RGB"
Graypath = '../Images/Jarrett_submissions2/Truthmaps_Gray'
with open(jsonpath) as json_file:
    json_data = json.load(json_file)
cmap_Ved = colors.ListedColormap(['xkcd:pink', 'xkcd:grey', 'xkcd:tan', 'xkcd:olive', 'xkcd:forest', 'xkcd:blue',
                                'xkcd:lilac', 'xkcd:cyan', 'xkcd:orange'])
coralutils.transform_RGB2Gray(finalpath_truthmap, Graypath, cmap_Ved, json_data["VedConsolidated_ClassDict"])

In [ ]:
""